In [2]:
!conda info


     active environment : subba
    active env location : /home2/akshett.jindal/miniconda3/envs/subba
            shell level : 2
       user config file : /home2/akshett.jindal/.condarc
 populated config files : /home2/akshett.jindal/.condarc
          conda version : 23.9.0
    conda-build version : not installed
         python version : 3.11.5.final.0
       virtual packages : __archspec=1=x86_64
                          __glibc=2.27=0
                          __linux=4.15.0=0
                          __unix=0=0
       base environment : /home2/akshett.jindal/miniconda3  (writable)
      conda av data dir : /home2/akshett.jindal/miniconda3/etc/conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/linux-64
                          https://repo.anaconda.com/pkgs/r/noarch
          package cache : /ho

In [44]:
!pip install -q scipy nibabel pandas

In [45]:
from nsd_gnet8x.src.file_utility import load_mask_from_nii, view_data
from nsd_gnet8x.src.roi import roi_map, iterate_roi
from nsd_gnet8x.src.load_nsd import image_feature_fn, ordering_split, load_betas
import scipy.io as sio
import numpy as np
import h5py
import pandas as pd

In [4]:
# ALL_SUBJECTS = [1, 2, 5, 7]
ALL_SUBJECTS = [1]

In [5]:
exp_design_file = "/tmp/akshett.jindal/nsd_expdesign.mat"
exp_design = sio.loadmat(exp_design_file)
exp_design.keys()

dict_keys(['__header__', '__version__', '__globals__', 'basiccnt', 'masterordering', 'sharedix', 'stimpattern', 'subjectim'])

In [6]:
ordering = exp_design["masterordering"].flatten() - 1
ordering.shape

(30000,)

In [7]:
basic_cnt    = exp_design['basiccnt']
shared_idx   = exp_design['sharedix']
subject_idx  = exp_design['subjectim']
trial_order  = exp_design['masterordering']
stim_pattern = exp_design['stimpattern']

In [33]:
print(f"{basic_cnt.shape = }, {basic_cnt.min() = }, {basic_cnt.max() = }")
print(f"{shared_idx.shape = }, {shared_idx.min() = }, {shared_idx.max() = }")
print(f"{subject_idx.shape = }, {subject_idx.min() = }, {subject_idx.max() = }")
print(f"{trial_order.shape = }, {trial_order.min() = }, {trial_order.max() = }")
print(f"{stim_pattern.shape = }, {stim_pattern.min() = }, {stim_pattern.max() = }")

basic_cnt.shape = (3, 40), basic_cnt.min() = 22, basic_cnt.max() = 465
shared_idx.shape = (1, 1000), shared_idx.min() = 2951, shared_idx.max() = 72949
subject_idx.shape = (8, 10000), subject_idx.min() = 1, subject_idx.max() = 73000
trial_order.shape = (1, 30000), trial_order.min() = 1, trial_order.max() = 10000
stim_pattern.shape = (40, 12, 75), stim_pattern.min() = 0, stim_pattern.max() = 1


In [41]:
(subject_idx[:,trial_order[0]-1]-1)

array([[46002, 61882,   828, ..., 53167,  1943,  5033],
       [46002, 42019, 22499, ..., 61375, 42647, 69767],
       [46002, 19256, 36385, ...,  5026, 53735, 29181],
       ...,
       [46002, 21396, 18924, ..., 48121, 63633, 17859],
       [46002, 16506,  9386, ..., 34540, 31243, 55230],
       [46002, 57483, 41827, ...,  4293, 23655, 29037]], dtype=int32)

In [ ]:
dataset_filepath = f"/tmp/akshett.jindal/nsddata_stimuli/nsd_stimuli_227.hdf5"
image_dataset = np.array(h5py.File(dataset_filepath, "r")['imgBrick'])

In [ ]:
BASE_DIRECTORY = "/tmp/akshett.jindal"

In [13]:
for subject in ALL_SUBJECTS:
    brain_mask_full = load_mask_from_nii(
        f"{BASE_DIRECTORY}/nsddata_brain_masks/subj{subject:02}/brainmask.nii.gz"
    )
    maskdata = load_mask_from_nii(
        f"{BASE_DIRECTORY}/nsddata_voxels/subj{subject:02}/nsdgeneral.nii.gz"
    )
    voxels = np.where(maskdata == 1)
    subject_image_ids = subject_idx[subject-1] - 1

    combined_session_data = None

    for session in range(1, 41):
        maindata = load_mask_from_nii(
            f"{BASE_DIRECTORY}/nsddata_sessions/subj{subject:02}/betas_session{session:02}.nii.gz"
        ).transpose(3, 0, 1, 2)

        # print(f"{maindata.shape = }")
        # print(f"{maskdata.shape = }")
        # print(f"{brain_mask_full.shape = }")

        current_session_data = maindata[:, voxels[0], voxels[1], voxels[2]]
        if combined_session_data is None:
            combined_session_data = current_session_data
        else:
            combined_session_data = np.concatenate((combined_session_data, current_session_data), axis=0)

    trn_stim_ordering, trn_voxel_data, val_stim_ordering, val_voxel_data = ordering_split(combined_session_data, ordering, combine_trial=False)
    print(f"{trn_stim_ordering.shape = }, {trn_voxel_data.shape = }")
    print(f"{val_stim_ordering.shape = }, {val_voxel_data.shape = }")

Total number of voxels = 15724
trn_stim_ordering.shape = (27000,), trn_voxel_data.shape = (27000, 15724)
val_stim_ordering.shape = (3000,), val_voxel_data.shape = (3000, 15724)


In [31]:
subject_image_ids = subject_idx[0] - 1
subject_image_ids.shape

(10000,)

In [27]:
image_dataset[subject_image_ids].shape

(10000, 227, 227, 3)